In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('/content/drive/MyDrive/A/gurgaon_properties_post_feature_selection.csv')

In [3]:
df.head()

,property_type,sector,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category,price
0,0.0,36.0,3.0,2.0,2.0,1.0,876.0,0.0,0.0,0.0,1.0,1.0,0.82
1,0.0,95.0,2.0,2.0,2.0,1.0,1226.0,1.0,0.0,0.0,1.0,2.0,0.95
2,0.0,103.0,2.0,2.0,1.0,1.0,1000.0,0.0,0.0,0.0,1.0,0.0,0.32
3,0.0,99.0,3.0,4.0,4.0,3.0,1615.0,1.0,0.0,1.0,0.0,2.0,1.60
4,0.0,5.0,2.0,2.0,1.0,3.0,597.0,0.0,1.0,0.0,0.0,2.0,0.48


In [4]:
# one hot encode -> sector, balcony, agePossession,furnishing,luxury category, floor category

In [5]:
x = df.drop(columns=['price'])
y = df['price']

In [34]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVR



In [8]:
columns_to_encode = ['sector', 'balcony', 'agePossession', 'furnishing_type', 'luxury_category', 'floor_category']

In [9]:
y_transformed = np.log1p(y)

In [44]:
# craeting a column transform for preprocessing
preprocessing = ColumnTransformer(
    transformers = [
    ('num', StandardScaler(), ['property_type', 'bedRoom','bathroom','built_up_area','servant room','store room']), # Fixed typo here
    ('cat', OneHotEncoder(drop='first'), columns_to_encode)
    ],
)

In [45]:
# creating a pipeline
pipeline = Pipeline([
    ('preprocessing', preprocessing),
    ('regressor', LinearRegression())
])

In [46]:
# k-fold cross-validation
# k-fold cross-validation
from sklearn.model_selection import KFold # make sure KFold is imported
kf = KFold(n_splits=10, shuffle=True, random_state=42) # change the variable name to kf
scores = cross_val_score(pipeline, x, y_transformed, cv=kf, scoring='r2') # use kf as the cv argument

In [47]:
scores.mean()

0.8563685822616567

In [48]:
scores.std()

0.015625810194415796

In [49]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y_transformed,test_size=0.2,random_state=42)

In [50]:
pipeline.fit(x_train,y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['property_type', 'bedRoom',
                                                   'bathroom', 'built_up_area',
                                                   'servant room',
                                                   'store room']),
                                                 ('cat',
                                                  OneHotEncoder(drop='first'),
                                                  ['sector', 'balcony',
                                                   'agePossession',
                                                   'furnishing_type',
                                                   'luxury_category',
                                                   'floor_category'])])),
                ('regressor', LinearRegression())])

In [51]:
y_pred = pipeline.predict(x_test)

In [52]:
y_pred = np.expm1(y_pred)

In [53]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(np.expm1(y_test),y_pred)

0.644667950251986

In [54]:
# # svm
# # creating a pipeline
# pipeline = Pipeline([
#     ('preprocessing', preprocessing),
#     ('regressor', SVR(kernel='rbf'))
# ])

# from sklearn.model_selection import KFold # make sure KFold is imported
# kf = KFold(n_splits=10, shuffle=True, random_state=42) # change the variable name to kf
# scores = cross_val_score(pipeline, x, y_transformed, cv=kf, scoring='r2') # use kf as the cv argument



In [55]:
scores.mean()

0.8563685822616567

In [56]:
scores.std()

0.015625810194415796